## Load Kofic API Movie List<br>

### kofic api 데이터 작업 순서<br>
- 영화목록 데이터 불러오기
- 데이터 로드 완료 후 load_movieDetails.ipynb 코드 실행
- 영화목록 데이터에 있는 movieCd를 이용해 영화상세목록 데이터 로드
- movies_merged.ipynb 코드 실행
- 영화목록과 상세정보 데이터 병합
- 상위 moviePoster폴더의 load_moviePoster.ipynb 코드 실행
- Naver api에서 포스터정보와 별점 정보를 로드
- movie_merge_poster.ipynb 코드 실행
- 데이터 병합

In [2]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

import math
import os
import json

In [2]:
api_key = "843263a42ef422fedcc5a1abcbbfbaa7"

In [1]:
save_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\movieInfo\all"

In [7]:
# 전부 가져오기
def load_movies():
    columns = ["movieCd", "movieNm", "movieNmEn", "prdtYear", "openDt", "typeNm", "prdtStatNm", 
               "nationAlt", "genreAlt", "repNationNm", "repGenreNm", "directors", "companyCd", "companyNm"]
    
    url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=430156241533f1d058c603178cc3ca0e&itemPerPage=1&curPage=1"
    res = requests.get(url)
    max_page = math.ceil(res.json()["movieListResult"]["totCnt"]/100)
    
    movie_list = []
    for page in tqdm_notebook(range(1, max_page+1)):            
        url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=430156241533f1d058c603178cc3ca0e&itemPerPage=100&curPage={page}"
        res = requests.get(url)
        max_page = math.ceil(res.json()["movieListResult"]["totCnt"]/100)
        
        movie_list.extend(res.json()["movieListResult"]["movieList"])

        df_movies = pd.DataFrame(movie_list)
        directors = pd.DataFrame([*df_movies.directors])

        temp = []
        for cols in directors:
            ls = []
            for i in directors[cols]:
                try:
                    ls.append(i["peopleNm"])
                except:
                    ls.append(" ")
            temp.append(ls)

        direc_values = list(zip(*temp))
        direc_values = list(map(lambda x: ", ".join(x).replace(",  ", ""), direc_values))
        df_movies.directors = direc_values
    
    companys = pd.DataFrame([*df_movies.companys])
    
    com_code = []
    com_name = []
    for cols in companys:
        ls1 = []
        ls2 = []
        for i in companys[cols]:
            try:
                ls1.append(i["companyCd"])
                ls2.append(i["companyNm"])
            except:
                ls1.append(" ")
                ls2.append(" ")
        com_code.append(ls1)
        com_name.append(ls2)
        
    code = list(zip(*com_code))
    name = list(zip(*com_name))
    
    df_movies["companyCd"] = code
    df_movies["companyNm"] = name
    df_movies = df_movies.drop("companys", axis=1)

    return df_movies[columns]

In [8]:
movies = load_movies()

In [46]:
len(movies)

72806

In [29]:
movies.genreAlt[:5]

0           드라마
1    드라마,멜로/로맨스
2           코미디
3       성인물(에로)
4       성인물(에로)
Name: genreAlt, dtype: object

In [47]:
# 성인영화 제거
av_idx = movies[movies.genreAlt.str.contains('성인')].index
av_idx

Int64Index([    3,     4,   258,   342,   359,   361,   373,   374,   396,
              397,
            ...
            71889, 71890, 71891, 71892, 71893, 71894, 71895, 71896, 71897,
            71898],
           dtype='int64', length=2302)

In [49]:
movies_dropAv = movies.drop(av_idx, axis=0)

In [59]:
mlist = movies_dropAv.copy()

In [60]:
id(mlist), id(movies_dropAv)

(1846921005768, 1846920731448)

In [61]:
mlist.tail()

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companyCd,companyNm
72801,20050294,CJ 아시아인디영화제[2005.11.3-11.8],,2005,20051103,기타,기타,,,,,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"
72802,20050243,제2회 서울환경영화제[2005.9.8-9.14],2nd Green Film Festival in Seoul,2005,20050908,기타,기타,,,,,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"
72803,20020194,레슨/달이지고 비가 옵니다/저푸른 초원(단편),,2002,20020719,단편,기타,한국,,한국,,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"
72804,20020310,엔조이/ 특집! 노래자랑(단편),,2001,20020913,단편,기타,한국,,한국,,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"
72805,19910096,칙칙이의 내일은 참피온,Tomorrow's Champion,1991,19910803,장편,개봉,한국,코미디,한국,코미디,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"


In [74]:
mlist.prdtStatNm.unique()  

array(['개봉', '개봉예정', '기타', '후반작업', '개봉준비', '촬영준비', '촬영진행', ''],
      dtype=object)

### prdtStatNm 데이터<br>
- "기타", "" 제거
- ~ 2015년 "개봉예정", "후반작업", "개봉준비", "촬영준비", "촬영진행" 제거

In [68]:
mlist_etc = mlist[mlist.prdtStatNm == "기타"]

In [144]:
mlist_etc.tail()

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companyCd,companyNm
72800,20050266,제6회 서울유럽영화제[2005.10.26-10.30],,2005,20051026,기타,기타,,,,,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"
72801,20050294,CJ 아시아인디영화제[2005.11.3-11.8],,2005,20051103,기타,기타,,,,,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"
72802,20050243,제2회 서울환경영화제[2005.9.8-9.14],2nd Green Film Festival in Seoul,2005,20050908,기타,기타,,,,,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"
72803,20020194,레슨/달이지고 비가 옵니다/저푸른 초원(단편),,2002,20020719,단편,기타,한국,,한국,,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"
72804,20020310,엔조이/ 특집! 노래자랑(단편),,2001,20020913,단편,기타,한국,,한국,,,"( , , , , , , , , , , , , , )","( , , , , , , , , , , , , , )"


In [139]:
idx = []
idx.extend(mlist[mlist.prdtStatNm == ""].index)
idx.extend(mlist[mlist.prdtStatNm == "기타"].index)
idx.extend(mlist[mlist.prdtStatNm == "개봉예정"][mlist.prdtYear < "2016"].index)
idx.extend(mlist[mlist.prdtStatNm == "후반작업"][mlist.prdtYear < "2016"].index)
idx.extend(mlist[mlist.prdtStatNm == "개봉준비"][mlist.prdtYear < "2016"].index)
idx.extend(mlist[mlist.prdtStatNm == "촬영준비"][mlist.prdtYear < "2016"].index)
idx.extend(mlist[mlist.prdtStatNm == "촬영진행"][mlist.prdtYear < "2016"].index)

C:\Users\Jwp\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
C:\Users\Jwp\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\Jwp\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\Jwp\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys
C:\Users\Jwp\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [141]:
len(mlist), len(idx), len(mlist) -len(idx)

(20410, 208, 20202)

In [142]:
mlist = mlist.drop(idx, axis=0)

In [97]:
len(mlist)

19874

### prdtYear, openDt, directors가 NaN인 데이터 제거

In [ ]:
idx = mlist[mlist.openDt.isnull()].index
mlist = mlist.drop(idx, axis=0)
idx = mlist[mlist.prdtYear.isnull()].index
mlist = mlist.drop(idx, axis=0)
idx = mlist[mlist.directors.isnull()].index
mlist = mlist.drop(idx, axis=0)

In [98]:
mlist.tail()

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companyCd,companyNm
20197,19910228,LA이야기,La Story,1991.0,19910608.0,장편,개봉,미국,코미디,미국,코미디,믹 잭슨,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."
20198,19920137,우리는 그들을 잊으려 했다,THE FORGOTTEN,1992.0,19920307.0,NaN,개봉,NaN,NaN,NaN,NaN,,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."
20199,19880146,원+씩스,ONE + SIX,1988.0,19880625.0,NaN,개봉,이탈리아,NaN,이탈리아,NaN,,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."
20200,19910263,최가손우2,CRAZY COMPANIES 2,1988.0,19910803.0,NaN,개봉,NaN,NaN,NaN,NaN,,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."
20201,19910096,칙칙이의 내일은 참피온,Tomorrow's Champion,1991.0,19910803.0,장편,개봉,한국,코미디,한국,코미디,,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."


### 날짜를 문자열로 변환

In [ ]:
mlist.openDt = mlist.openDt.astype(int).astype(str)
mlist.prdtYear = mlist.prdtYear.astype(int).astype(str)

In [107]:
mlist.openDt[0]

'20190905'

In [108]:
mlist.to_csv(os.path.join(save_path, "movie_list_all.csv"), encoding="utf-8", index=None)

In [57]:
# mlist.to_csv(os.path.join(save_path, "movie_list_all_euc.csv"), encoding="euc-kr", index=None)

In [64]:
mlist = pd.read_csv(os.path.join(save_path, "movie_list_all.csv"), encoding="utf-8")

In [70]:
len(mlist)

20202

In [87]:
mlist

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companyCd,companyNm
0,20196285,오만,Huris,2019.0,20190905.0,장편,개봉,한국,드라마,한국,드라마,지일주,"('20156848', ' ', ' ', ' ', ' ', ' ', ' ', ' '...","('(주)백그림', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."
1,20183407,버티고,Vertigo,2019.0,20191016.0,장편,개봉,한국,"드라마,멜로/로맨스",한국,드라마,전계수,"('20100414', ' ', ' ', ' ', ' ', ' ', ' ', ' '...","('영화사 도로시(주)', ' ', ' ', ' ', ' ', ' ', ' ', '..."
3,20199428,크롤,CRAWL,2019.0,20191127.0,장편,개봉예정,미국,"액션,스릴러",미국,액션,알렉산더 아야,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."
4,20191029,82년생 김지영,"KIM JI-YOUNG, BORN 1982",2019.0,20191023.0,장편,개봉,한국,드라마,한국,드라마,김도영,"('20192181', ' ', ' ', ' ', ' ', ' ', ' ', ' '...","('(주)봄바람영화사', ' ', ' ', ' ', ' ', ' ', ' ', ' ..."
7,20149501,사선의 끝,The Threshold of Death,2019.0,20191030.0,장편,개봉예정,한국,드라마,한국,드라마,이은상,"('20124237', ' ', ' ', ' ', ' ', ' ', ' ', ' '...","('(주)엠씨엠씨', ' ', ' ', ' ', ' ', ' ', ' ', ' ',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20197,19910228,LA이야기,La Story,1991.0,19910608.0,장편,개봉,미국,코미디,미국,코미디,믹 잭슨,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."
20198,19920137,우리는 그들을 잊으려 했다,THE FORGOTTEN,1992.0,19920307.0,NaN,개봉,NaN,NaN,NaN,NaN,,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."
20199,19880146,원+씩스,ONE + SIX,1988.0,19880625.0,NaN,개봉,이탈리아,NaN,이탈리아,NaN,,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."
20200,19910263,최가손우2,CRAZY COMPANIES 2,1988.0,19910803.0,NaN,개봉,NaN,NaN,NaN,NaN,,"(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ...","(' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ..."


0        2019
1        2019
3        2019
4        2019
7        2019
         ... 
20197    1991
20198    1992
20199    1988
20200    1988
20201    1991
Name: prdtYear, Length: 19874, dtype: object

In [10]:
url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=430156241533f1d058c603178cc3ca0e&itemPerPage=100&curPage={page}"
res = requests.get(url)
res.json()["movieListResult"]["movieList"]

[{'movieCd': '20191029',
  'movieNm': '82년생 김지영',
  'movieNmEn': 'KIM JI-YOUNG, BORN 1982 ',
  'prdtYear': '2019',
  'openDt': '20191023',
  'typeNm': '장편',
  'prdtStatNm': '개봉',
  'nationAlt': '한국',
  'genreAlt': '드라마',
  'repNationNm': '한국',
  'repGenreNm': '드라마',
  'directors': [{'peopleNm': '김도영'}],
  'companys': [{'companyCd': '20192181', 'companyNm': '(주)봄바람영화사'}]},
 {'movieCd': '20198681',
  'movieNm': '쥬만지: 넥스트 레벨',
  'movieNmEn': 'Jumanji : The Next Level',
  'prdtYear': '2019',
  'openDt': '',
  'typeNm': '장편',
  'prdtStatNm': '개봉예정',
  'nationAlt': '미국',
  'genreAlt': '액션,어드벤처',
  'repNationNm': '미국',
  'repGenreNm': '액션',
  'directors': [{'peopleNm': '제이크 캐스단'}],
  'companys': []},
 {'movieCd': '20199880',
  'movieNm': '잃어버린 낙원의 꿈',
  'movieNmEn': 'The Lost Dream of a Paradise',
  'prdtYear': '2017',
  'openDt': '',
  'typeNm': '단편',
  'prdtStatNm': '기타',
  'nationAlt': '한국',
  'genreAlt': '다큐멘터리',
  'repNationNm': '한국',
  'repGenreNm': '다큐멘터리',
  'directors': [],
  'compan

In [64]:
# 개봉연도로 데이터 조회
def load_movies(year, end_page):
    Year = year
    PageN = range(1, end_page+1)
    columns = ["movieCd", "movieNm", "movieNmEn", "prdtYear", "openDt", "typeNm", 
        "prdtStatNm", "nationAlt", "genreAlt", "repNationNm", "repGenreNm", "directors", "companyCd", "companyNm"]
    
    movie_list = []
    for page in PageN:
        url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=430156241533f1d058c603178cc3ca0e&openStartDt={Year}&openEndDt={Year}&itemPerPage=100&curPage={page}"
        res = requests.get(url)
        movie_list.extend(res.json()["movieListResult"]["movieList"])
    
        df_movies = pd.DataFrame(movie_list)
        directors = pd.DataFrame([*df_movies.directors])

        temp = []
        for cols in directors:
            ls = []
            for i in directors[cols]:
                try:
                    ls.append(i["peopleNm"])
                except:
                    ls.append(" ")
            temp.append(ls)

        direc_values = list(zip(*temp))
        direc_values = list(map(lambda x: ", ".join(x).replace(",  ", ""), direc_values))
        df_movies.directors = direc_values
    
    companys = pd.DataFrame([*df_movies.companys])
    
    com_code = []
    com_name = []
    for cols in companys:
        ls1 = []
        ls2 = []
        for i in companys[cols]:
            try:
                ls1.append(i["companyCd"])
                ls2.append(i["companyNm"])
            except:
                ls1.append(" ")
                ls2.append(" ")
        com_code.append(ls1)
        com_name.append(ls2)
        
    code = list(zip(*com_code))
    name = list(zip(*com_name))
    
    df_movies["companyCd"] = code
    df_movies["companyNm"] = name
    df_movies = df_movies.drop("companys", axis=1)

    return df_movies[columns]

In [66]:
df_movies = load_movies(2018, 19)

In [69]:
len(df_movies.columns)

14

In [ ]:
df_movies.to_csv(os.path.join(save_path, "movies_2018_3.csv"), encoding="utf-8", index=None)

In [ ]:
df_movies.to_csv(os.path.join(save_path, "movies_2018_3_euc.csv"), encoding="euc-kr", index=None)

In [48]:
companys = pd.DataFrame([*df_movies.companys])

In [49]:
companys.head()

,0,1,2
0,"{'companyCd': '20186561', 'companyNm': '(주)필름케이'}","{'companyCd': '20100374', 'companyNm': '(주)외유내강'}",None
1,None,None,None
2,"{'companyCd': '20101149', 'companyNm': '(주)페퍼민...",None,None
3,"{'companyCd': '20100947', 'companyNm': '(주)태원엔...",None,None
4,"{'companyCd': '20112272', 'companyNm': '(주)파란프...",None,None


In [50]:
companys[companys[2].isnull() == False]

,0,1,2
174,"{'companyCd': '20100278', 'companyNm': '(주)제이케...","{'companyCd': '20188461', 'companyNm': '그리고픽처스'}","{'companyCd': '20110854', 'companyNm': '씨제이이앤엠..."
199,"{'companyCd': '20139999', 'companyNm': '(주)무비락'}","{'companyCd': '20140755', 'companyNm': '(주)도서관...","{'companyCd': '20173761', 'companyNm': '(주)푸른나..."
200,"{'companyCd': '20100123', 'companyNm': '(주)싸이더스'}","{'companyCd': '20124237', 'companyNm': '(주)엠씨엠씨'}","{'companyCd': '20062469', 'companyNm': '(주)더블앤..."
656,"{'companyCd': '20141515', 'companyNm': '(주)영화사...","{'companyCd': '20161781', 'companyNm': '(주)스튜디...","{'companyCd': '20140888', 'companyNm': '모티브픽쳐스..."
1262,"{'companyCd': '20157148', 'companyNm': '(주)케이프...","{'companyCd': '20101396', 'companyNm': '버티고필름'}","{'companyCd': '20123175', 'companyNm': '플로우식스(..."
1428,"{'companyCd': '20139800', 'companyNm': '영화사 이지'}","{'companyCd': '20186741', 'companyNm': '한국영상대학교'}","{'companyCd': '20173382', 'companyNm': '(주)컴퍼니..."


In [51]:
com_code = []
com_name = []
for cols in companys:
    ls1 = []
    ls2 = []
    for i in companys[cols]:
        try:
            ls1.append(i["companyCd"])
            ls2.append(i["companyNm"])
        except:
            ls1.append(" ")
            ls2.append(" ")
    com_code.append(ls1)
    com_name.append(ls2)

In [52]:
code = list(zip(*com_code))
name = list(zip(*com_name))

In [53]:
list(zip(*com_code, *com_name))[:5]

[('20186561', '20100374', ' ', '(주)필름케이', '(주)외유내강', ' '),
 (' ', ' ', ' ', ' ', ' ', ' '),
 ('20101149', ' ', ' ', '(주)페퍼민트앤컴퍼니', ' ', ' '),
 ('20100947', ' ', ' ', '(주)태원엔터테인먼트', ' ', ' '),
 ('20112272', ' ', ' ', '(주)파란프로덕션', ' ', ' ')]

In [54]:
name[:5]

[('(주)필름케이', '(주)외유내강', ' '),
 (' ', ' ', ' '),
 ('(주)페퍼민트앤컴퍼니', ' ', ' '),
 ('(주)태원엔터테인먼트', ' ', ' '),
 ('(주)파란프로덕션', ' ', ' ')]

In [55]:
df_movies["companyCd"] = code
df_movies["companyNm"] = name

In [56]:
list(zip(code, name))[:5]

[(('20186561', '20100374', ' '), ('(주)필름케이', '(주)외유내강', ' ')),
 ((' ', ' ', ' '), (' ', ' ', ' ')),
 (('20101149', ' ', ' '), ('(주)페퍼민트앤컴퍼니', ' ', ' ')),
 (('20100947', ' ', ' '), ('(주)태원엔터테인먼트', ' ', ' ')),
 (('20112272', ' ', ' '), ('(주)파란프로덕션', ' ', ' '))]

In [57]:
df_movies.head()

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companys,companyCd,companyNm
0,20153446,너의 결혼식,On Your Wedding Day,2017,20180822,장편,개봉,한국,멜로/로맨스,한국,멜로/로맨스,이석근,"[{'companyCd': '20186561', 'companyNm': '(주)필름...","(20186561, 20100374, )","((주)필름케이, (주)외유내강, )"
1,20182989,커다랗고 커다랗고 커다란 배,The Incredible Story of the Giant Pear,2017,20180503,장편,개봉,덴마크,애니메이션,덴마크,애니메이션,"필립 립스키, 요르겐 레르담",[],"( , , )","( , , )"
2,20174442,뷰티풀 데이즈,Beautiful Days,2018,20181121,장편,개봉,"한국,프랑스",드라마,한국,드라마,윤재호,"[{'companyCd': '20101149', 'companyNm': '(주)페퍼...","(20101149, , )","((주)페퍼민트앤컴퍼니, , )"
3,20184678,배반의 장미,Too Hot to Die,2018,20181018,장편,개봉,한국,코미디,한국,코미디,박진영,"[{'companyCd': '20100947', 'companyNm': '(주)태원...","(20100947, , )","((주)태원엔터테인먼트, , )"
4,20168766,종이 비행기,Paper Planes,2016,20180131,장편,개봉,한국,드라마,한국,드라마,"노홍식, 필동","[{'companyCd': '20112272', 'companyNm': '(주)파란...","(20112272, , )","((주)파란프로덕션, , )"


In [58]:
df_movies = df_movies.drop("companys", axis=1)

In [59]:
df_movies.head()

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companyCd,companyNm
0,20153446,너의 결혼식,On Your Wedding Day,2017,20180822,장편,개봉,한국,멜로/로맨스,한국,멜로/로맨스,이석근,"(20186561, 20100374, )","((주)필름케이, (주)외유내강, )"
1,20182989,커다랗고 커다랗고 커다란 배,The Incredible Story of the Giant Pear,2017,20180503,장편,개봉,덴마크,애니메이션,덴마크,애니메이션,"필립 립스키, 요르겐 레르담","( , , )","( , , )"
2,20174442,뷰티풀 데이즈,Beautiful Days,2018,20181121,장편,개봉,"한국,프랑스",드라마,한국,드라마,윤재호,"(20101149, , )","((주)페퍼민트앤컴퍼니, , )"
3,20184678,배반의 장미,Too Hot to Die,2018,20181018,장편,개봉,한국,코미디,한국,코미디,박진영,"(20100947, , )","((주)태원엔터테인먼트, , )"
4,20168766,종이 비행기,Paper Planes,2016,20180131,장편,개봉,한국,드라마,한국,드라마,"노홍식, 필동","(20112272, , )","((주)파란프로덕션, , )"
